# Inverse Kinematics Stage

In [1]:
"""
Created on Wed Feb  7 09:23:56 2024

@author: parallel
"""
import numpy as np


def invKin(pos):
    """
    enter x, y, z and output theta vector 
    
    """
    # define robot dimentions
    x, y, z = pos[0], pos[1], pos[2]
    w_b = 80
    s_b = 277.128
    u_b = 160

    w_p = 15.16
    s_p = 52.5
    u_p = 30.31

    L = 120
    l = 340
    h = 55
    
    # define parameters
    a = w_b - u_p
    b = s_p / 2 - (np.sqrt(3)/2) * w_b
    c = w_p - w_b / 2
    
    # calc
    E1 = 2*L*(y+a)
    F1 = 2*z*L
    G1 = x**2 + y**2 + z**2 + a**2 + L**2 + 2*y*a - l**2
    
    E2 = -L*(np.sqrt(3) * (x+b) + y + c)
    F2 = 2*z*L
    G2 = x**2 + y**2 + z**2 + b**2 + c**2 + L**2 + 2*(x*b + y*c) - l**2
    
    E3 = L*(np.sqrt(3)*(x-b)-y-c)
    F3 = 2*z*L
    G3 = x**2 + y**2 + z**2 + b**2 + c**2 + L**2 + 2*(-x*b + y*c) - l**2
    
    t1_p = (-F1 + np.sqrt(E1**2 + F1**2 - G1**2)) / (G1 - E1)
    t1_m = (-F1 - np.sqrt(E1**2 + F1**2 - G1**2)) / (G1 - E1)
    t2_p = (-F2 + np.sqrt(E2**2 + F2**2 - G2**2)) / (G2 - E2)
    t2_m = (-F2 - np.sqrt(E2**2 + F2**2 - G2**2)) / (G2 - E2)
    t3_p = (-F3 + np.sqrt(E3**2 + F3**2 - G3**2)) / (G3 - E3)
    t3_m = (-F3 - np.sqrt(E3**2 + F3**2 - G3**2)) / (G3 - E3) 
    
    # get theta
    theta1_p = 2*np.arctan2(t1_m*(G1 - E1), (G1 - E1))
    theta2_p = 2*np.arctan2(t2_m*(G2 - E2), (G2 - E2))
    theta3_p = 2*np.arctan2(t3_m*(G3 - E3), (G3 - E3))
#    theta1_m = 2*np.arctan2(t1_m)
#    theta1_m = 2*np.arctan2(t2_m)
#    theta1_m = 2*np.arctan2(t3_m)

        
    theta = np.array([theta1_p, theta2_p, theta3_p]) * 180 / np.pi
    
    # make sure the range output is between -30 to 60 degrees.
    for inx in range(3):
        print("h")
        if theta[inx] >= 330 and theta[inx] <= 360: # correct part from -30 to 0
            theta[inx] -= 360
            
        elif theta[inx] >= -360 and theta[inx] <= -300: # correct the part correesponding to 0 to 60
            theta[inx] += 360
        
        
        elif theta[inx] > 60 or theta[inx] < -30:
            print(f"theta vec is out of range - it is = {theta}")
            print("return to home because you suck")
            return [-30, -30, -30]
    return theta



# Coordinate System Transformations

In [2]:
def cs_cage_to_base(cage_position):
    """
    The location of base w.r.t cage coordinate system is:
        x = 183
        y = 262
        z = 405
    The rotation of base w.r.t cage coordinate system is:
        61 degrees about z axis = 1.064650843716541 rad
        -R_T@d = [140.43020277 -287.07552691 -405]
    """
    ang = 1.064650843716541
    
    M = np.array([
        [np.cos(ang), -np.sin(ang), 0, 140.43020277],
        [np.sin(ang), np.cos(ang), 0, -287.07552691],
        [0, 0, 1, -405],
        [0, 0, 0, 1]
    ])
    base_position = M @ np.array(cage_position + [1])
    return base_position[:3]

   

In [1]:
def cs_frame_to_cage(x, y, h, w):
    x_mid = x + w/2 + 93
    y_mid = - y + h/2 + 288
    cage_coords_to_pick = [x_mid, y_mid, 20]
    return cage_coords_to_pick


# Calibration Stage

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 14 09:36:01 2024

@author: parallel
"""

# calibrate camera
# Import required modules
import cv2
import numpy as np
import os
import glob


def capture_image():

    #  Rotation Vectors:
    # (array([[ 0.08488239],
    #        [-0.00621647],
    #        [-0.01404018]]),)

    #  Translation Vectors:
    # (array([[-11.94359699],
    #        [ -0.39084338],
    #        [ 48.83364547]]),)
      
    video = cv2.VideoCapture(0)
    # get image from camera(corrent frame!)
    ret, frame = video.read()

    matrix = np.array(
        [[1.17673043e+03, 0.00000000e+00, 3.22412128e+02],
         [0.00000000e+00, 1.18810840e+03, 1.74915728e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

    distortion = np.array(
        [[ 4.25806110e-01, -2.72307004e+00, -7.51942459e-03, -3.68742702e-03,
          -5.69413505e+00]])


    img = frame
    h,  w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(matrix, distortion, (w,h), 1, (w,h))

    # undistort
    mapx, mapy = cv2.initUndistortRectifyMap(matrix, distortion, None, newcameramtx, (w,h), 5)
    dst = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv2.imwrite('dst.png', dst)

    # disconnect from camera
    video.release()
    
    # # wait for a key event and close the windows
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()               
                  
    return 'dst.png'


def find_block(filename, color):
    image = cv2.imread(filename)

    # # plot image 
    # cv2.imshow('Image', image)
      
    # change image format to HSV
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    if color == 'yellow':
        lower = np.array([17, 141, 150])
        upper = np.array([37, 255, 255])
    elif color == 'blue':
        lower = np.array([77, 177, 73])
        upper = np.array([179, 255, 255])
    elif color == 'green':
        lower = np.array([34, 81, 0])
        upper = np.array([61, 255, 255])
    elif color == 'orange':
        lower = np.array([0, 141, 150])
        upper = np.array([15, 255, 255])
    
    # Create a mask for the color
    mask = cv2.inRange(image, lower, upper)

    # check mask:
    # cv2.imshow('mask', mask)
    cv2.imwrite(f'Mask_{color}.png', mask)

    # Find the contours in the mask
    contours, _  = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    largest_contour = max(contours, key=cv2.contourArea)
    # Draw a rectangle around the contour
    x, y, w, h = cv2.boundingRect(largest_contour)
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the image
    # cv2.imshow('Image', image)
    cv2.imwrite(f'Rect_{color}.png', image)
    print(color)
    print("h = ", h)
    print("w = ", w)
    print("x = ", x)
    print("y = ", y)

    return  x, y, w, h


# find_block(capture_image(), 'yellow')
# find_block(capture_image(), 'green')
# find_block(capture_image(), 'blue')
# find_block(capture_image(), 'orange')
 